In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [21]:
bicing_status_cols = ['last_updated', 'station_id', 'num_bikes_available', 'num_docks_available']
bicing_status_2212 = pd.read_csv(r'..\data\raw\bicing_stations_status\2022\2022_12_Desembre_BicingNou_ESTACIONS.csv', usecols=bicing_status_cols)

In [22]:
bicing_status_2212.sample(20)

,station_id,num_bikes_available,num_docks_available,last_updated
4016220,221,17,7,1672234186
3023806,50,16,2,1671645901
2289806,304,2,24,1671209694
2553660,49,20,0,1671366580
2795601,240,26,0,1671510283
3432719,118,1,26,1671888277
1544349,184,5,22,1670766883
838520,265,6,21,1670347475
3547456,507,21,2,1671956079
1603018,274,17,15,1670801684


In [23]:
bicing_status_2212.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4512523 entries, 0 to 4512522
Data columns (total 4 columns):
 #   Column               Dtype
---  ------               -----
 0   station_id           int64
 1   num_bikes_available  int64
 2   num_docks_available  int64
 3   last_updated         int64
dtypes: int64(4)
memory usage: 137.7 MB


In [24]:
bicing_status_2212.shape

(4512523, 4)

In [25]:
bicing_status_2212.describe().T

,count,mean,std,min,25%,50%,75%,max
station_id,4512523.0,2.615072e+02,149.427218,1.000000e+00,1.320000e+02,2.630000e+02,3.900000e+02,5.190000e+02
num_bikes_available,4512523.0,1.072229e+01,7.664918,0.000000e+00,5.000000e+00,1.000000e+01,1.600000e+01,5.400000e+01
num_docks_available,4512523.0,1.566360e+01,7.858505,0.000000e+00,1.000000e+01,1.600000e+01,2.100000e+01,5.400000e+01
last_updated,4512523.0,1.671189e+09,773538.357338,1.669849e+09,1.670520e+09,1.671190e+09,1.671859e+09,1.672528e+09


In [26]:
bicing_status_2212.memory_usage(deep=True).sum()

144400864

In [27]:
bicing_status_2212.isna().sum()

station_id             0
num_bikes_available    0
num_docks_available    0
last_updated           0
dtype: int64

In [81]:
def tweak_bicing_status(df):
    weekend = {1: False, 2: False, 3: False, 4: False, 5: False, 6: True, 7: True}
    
    def int64_to_uint16(df_):
        cols = df_.select_dtypes('int64')
        return (df_
                .astype({col:'uint16' for col in cols}))
    return (df
            .assign(last_updated=pd.to_datetime(df['last_updated'], unit='s'), 
                    year=lambda df_: df_['last_updated'].dt.year.astype('uint16'),
                    month=lambda df_: df_['last_updated'].dt.month.astype('uint8'),
                    day=lambda df_: df_['last_updated'].dt.day.astype('uint8'),
                    weekday=lambda df_: df_['last_updated'].dt.isocalendar().day.astype('uint8'),
                    hour=lambda df_: df_['last_updated'].dt.hour.astype('uint8'),
                    weekend=lambda df_: df_['weekday'].map(weekend),
                    percentage_docks_available=lambda df_: df_['num_docks_available'].div(df_['num_docks_available'].add(df_['num_bikes_available'])))
            .pipe(int64_to_uint16)
        #     .drop(columns='last_updated')
        #     .drop_duplicates()
    )

In [85]:
tweak_bicing_status(bicing_status_2212).groupby(['station_id', 'year', 'month', 'hour'], as_index=False).agg({'num_bikes_available': 'unique', 'num_docks_available': 'unique'}).query('station_id == 277')

,station_id,year,month,hour,num_bikes_available,num_docks_available
6916,277,2022,11,22,[26],[0]
6917,277,2022,11,23,"[26, 25, 24]","[0, 1, 2]"
6918,277,2022,12,0,"[25, 0, 27, 24, 26, 22, 23, 7, 6, 8, 9, 13, 14...","[1, 0, 3, 2, 5, 4, 20, 21, 19, 18, 14, 13, 6, ..."
6919,277,2022,12,1,"[25, 27, 26, 23, 24, 9, 10, 11, 14, 13, 19, 21...","[1, 0, 2, 4, 3, 18, 17, 16, 13, 14, 8, 6, 5, 1..."
6920,277,2022,12,2,"[25, 26, 27, 24, 23, 11, 12, 13, 14, 15, 22, 2...","[1, 0, 2, 3, 4, 16, 15, 14, 13, 12, 5, 7, 9, 2..."
6921,277,2022,12,3,"[25, 20, 27, 24, 26, 23, 11, 15, 19, 7, 8, 9, ...","[1, 6, 0, 7, 3, 2, 4, 16, 12, 8, 19, 18, 5, 14..."
6922,277,2022,12,4,"[20, 19, 26, 27, 25, 18, 23, 22, 11, 13, 15, 1...","[6, 7, 8, 1, 0, 2, 9, 4, 5, 16, 14, 12, 13, 18..."
6923,277,2022,12,5,"[19, 18, 26, 25, 24, 22, 21, 13, 14, 12, 11, 1...","[7, 8, 1, 2, 3, 9, 5, 6, 14, 13, 15, 16, 10, 1..."
6924,277,2022,12,6,"[18, 17, 16, 14, 12, 11, 10, 9, 19, 15, 24, 22...","[8, 9, 10, 12, 14, 15, 16, 17, 11, 13, 3, 5, 4..."
6925,277,2022,12,7,"[10, 9, 7, 6, 5, 1, 15, 14, 11, 12, 23, 22, 20...","[16, 17, 19, 18, 20, 21, 25, 12, 13, 15, 4, 5,..."


In [53]:
tweak_bicing_status(bicing_status_2212)[tweak_bicing_status(bicing_status_2212)['percentage_docks_available'].isna()]

,station_id,num_bikes_available,num_docks_available,year,month,day,weekday,hour,weekend,percentage_docks_available
105637,352,0,0,2022,12,1,4,16,False,NaN
110402,60,0,0,2022,12,1,4,17,False,NaN
116167,277,0,0,2022,12,1,4,18,False,NaN
121722,277,0,0,2022,12,1,4,19,False,NaN
127782,277,0,0,2022,12,1,4,20,False,NaN
...,...,...,...,...,...,...,...,...,...,...
3090225,190,0,0,2022,12,22,4,5,False,NaN
3096297,190,0,0,2022,12,22,4,6,False,NaN
3102875,190,0,0,2022,12,22,4,7,False,NaN
3108441,190,0,0,2022,12,22,4,8,False,NaN


In [80]:
(tweak_bicing_status(bicing_status_2212)
 .query('month == 12')
 .drop_duplicates(subset=['station_id', 'year', 'month', 'day', 'hour'])
 .sort_values(by=['station_id', 'year', 'month', 'day', 'hour'])
 .assign(ctx_1=tweak_bicing_status(bicing_status_2212).groupby('station_id')['percentage_docks_available'].shift(1),
         ctx_2=tweak_bicing_status(bicing_status_2212).groupby('station_id')['percentage_docks_available'].shift(2),
         ctx_3=tweak_bicing_status(bicing_status_2212).groupby('station_id')['percentage_docks_available'].shift(3),
         ctx_4=tweak_bicing_status(bicing_status_2212).groupby('station_id')['percentage_docks_available'].shift(4),)
 .dropna()
)#.groupby('month', as_index=False).agg({'hour': ['nunique', 'unique']})

,station_id,num_bikes_available,num_docks_available,year,month,day,weekday,hour,weekend,percentage_docks_available,ctx_1,ctx_2,ctx_3,ctx_4
6565,1,29,17,2022,12,1,4,0,False,0.369565,0.369565,0.434783,0.391304,0.413043
12613,1,32,14,2022,12,1,4,1,False,0.304348,0.304348,0.326087,0.347826,0.369565
18637,1,32,14,2022,12,1,4,2,False,0.304348,0.282609,0.304348,0.304348,0.326087
24671,1,32,14,2022,12,1,4,3,False,0.304348,0.304348,0.282609,0.304348,0.304348
30707,1,32,14,2022,12,1,4,4,False,0.304348,0.304348,0.304348,0.282609,0.304348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4487227,519,4,20,2022,12,31,6,19,True,0.833333,0.875000,0.833333,0.958333,0.916667
4493294,519,7,17,2022,12,31,6,20,True,0.708333,0.708333,0.750000,0.833333,0.875000
4499366,519,9,15,2022,12,31,6,21,True,0.625000,0.625000,0.708333,0.708333,0.750000
4505438,519,10,14,2022,12,31,6,22,True,0.583333,0.583333,0.541667,0.625000,0.625000


In [34]:
tweak_bicing_status(bicing_status_2212).info()

<class 'pandas.core.frame.DataFrame'>
Index: 1286126 entries, 0 to 4512515
Data columns (total 10 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   station_id                  1286126 non-null  uint16 
 1   num_bikes_available         1286126 non-null  uint16 
 2   num_docks_available         1286126 non-null  uint16 
 3   year                        1286126 non-null  uint16 
 4   month                       1286126 non-null  uint8  
 5   day                         1286126 non-null  uint8  
 6   weekday                     1286126 non-null  uint8  
 7   hour                        1286126 non-null  uint8  
 8   weekend                     1286126 non-null  bool   
 9   percentage_docks_available  1286050 non-null  float64
dtypes: bool(1), float64(1), uint16(4), uint8(4)
memory usage: 35.6 MB


In [35]:
tweak_bicing_status(bicing_status_2212).head()

,station_id,num_bikes_available,num_docks_available,year,month,day,weekday,hour,weekend,percentage_docks_available
0,1,29,15,2022,11,30,3,22,False,0.340909
1,2,18,10,2022,11,30,3,22,False,0.357143
2,3,11,16,2022,11,30,3,22,False,0.592593
3,4,13,8,2022,11,30,3,22,False,0.380952
4,5,13,26,2022,11,30,3,22,False,0.666667


In [12]:
def load_table(year, month, month_name):
    """Loads the bicing stations data from a CSV file.

    Args:
        year (int): The year of the data.
        month (int): The month of the data.
        month_name (str): The name of the month in Catalan.

    Returns:
        DataFrame: A Pandas DataFrame containing the bicing stations data with columns:
            - last_updated: Timestamp of the last update
            - station_id: ID of the station
            - num_bikes_available: Number of available bikes at the station
            - num_docks_available: Number of available docks at the station
    """
    bicing_status_cols = ['last_updated', 'station_id', 'num_bikes_available', 'num_docks_available']
    return pd.read_csv(fr'..\data\raw\bicing_stations_status\{year}\{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.csv', usecols=bicing_status_cols)

In [15]:
i2m = list(zip(range(1, 13), ['Gener', 'Febrer', 'Març', 'Abril', 'Maig', 'Juny', 'Juliol', 'Agost', 'Setembre', 'Octubre', 'Novembre', 'Desembre']))

bicing_status = pd.concat([tweak_bicing_status(load_table(year, month, month_name)) for year in tqdm([2022]) for month, month_name in tqdm(i2m)])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [16]:
bicing_status.head()

,station_id,num_bikes_available,num_docks_available,year,month,day,weekday,hour,weekend
0,1,18,28,2021,12,31,5,22,False
1,2,18,11,2021,12,31,5,22,False
2,3,12,15,2021,12,31,5,22,False
3,4,2,18,2021,12,31,5,22,False
4,5,6,33,2021,12,31,5,22,False


In [17]:
bicing_status.shape

(15964196, 9)

In [18]:
bicing_status.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15964196 entries, 0 to 4512515
Data columns (total 9 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   station_id           uint16
 1   num_bikes_available  uint16
 2   num_docks_available  uint16
 3   year                 uint16
 4   month                uint8 
 5   day                  uint8 
 6   weekday              uint8 
 7   hour                 uint8 
 8   weekend              bool  
dtypes: bool(1), uint16(4), uint8(4)
memory usage: 319.7 MB


In [19]:
bicing_status.describe().T

,count,mean,std,min,25%,50%,75%,max
station_id,15964196.0,243.491658,149.049375,1.0,112.0,232.0,373.0,532.0
num_bikes_available,15964196.0,9.753900,7.631991,0.0,3.0,8.0,15.0,54.0
num_docks_available,15964196.0,15.389999,8.019479,0.0,10.0,16.0,21.0,54.0
year,15964196.0,2021.999870,0.011408,2021.0,2022.0,2022.0,2022.0,2022.0
month,15964196.0,6.688939,3.378110,1.0,4.0,7.0,10.0,12.0
day,15964196.0,15.763246,8.811884,1.0,8.0,16.0,23.0,31.0
weekday,15964196.0,3.937701,1.958876,1.0,2.0,4.0,6.0,7.0
hour,15964196.0,12.655298,6.270514,0.0,8.0,13.0,18.0,23.0


In [61]:
bicing_status[(bicing_status['num_docks_available'] == 0) & (bicing_status['num_bikes_available'] == 0)].station_id.unique()

array([287, 411, 125, 373,  98, 123, 224, 378,  89, 148,   2, 425, 342,
       211, 275, 458,  61, 381, 129, 218, 109,  80, 455, 166,  50, 441,
        34, 153, 145, 410, 284, 333, 222, 113,  32,  62, 187, 433, 316,
        28, 496, 180, 106, 152, 280, 220, 401,  84, 182, 114, 454, 212,
       302, 168, 149, 191,  11,  77, 206, 400,  79, 128, 376, 192, 276,
        31, 448, 111, 185, 119, 154,  37, 395, 124,   9,  56, 175, 105,
        81,  33, 291, 171, 427, 438, 103, 127, 160, 437,  12,  65, 121,
       221, 388, 144, 506, 225,  88, 355, 331, 322, 340, 397,  40, 315,
       226, 272, 104, 126, 161, 503, 351, 415, 366,  54,  53,  21,   4,
       243, 176, 207, 431, 527, 383,  20, 183, 447, 232, 164,  57, 424,
        90, 117,  41, 463, 190,  42, 155, 133,  60, 500,   3,  46, 390,
       186, 354, 449, 446,  75, 359,  69,  71, 374, 393, 343, 386, 409,
       245, 165, 422, 319, 237, 430, 357,  36, 406, 429, 307, 112,  87,
       102, 428, 505, 502, 398, 151, 143,  45, 426,  27,  44, 19

In [60]:
bicing_status.groupby('station_id', as_index=False).agg({'month': 'nunique'})#.query('month != 12')

,station_id,month
91,96,8
155,160,8
183,191,11
279,287,11
411,420,8
422,431,6
499,509,11
510,520,1
511,527,1
512,532,1


In [79]:
bicing_status['station_id'].nunique()

513